In [1]:
import pandas as pd
import numpy as np

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_csv("/content/drive/MyDrive/study/group-HW-fraud/financial_loan.csv")

In [11]:
print("Размер данных:", df.shape)

Размер данных: (38576, 24)


In [12]:
print("\nПример строк данных:")
display(df.head(5))


Пример строк данных:


,id,address_state,application_type,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,...,sub_grade,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment
0,1077430,GA,INDIVIDUAL,< 1 year,Ryder,C,RENT,11-02-2021,13-09-2021,13-04-2021,...,C4,60 months,Source Verified,30000.0,0.0100,59.83,0.1527,2500,4,1009
1,1072053,CA,INDIVIDUAL,9 years,MKC Accounting,E,RENT,01-01-2021,14-12-2021,15-01-2021,...,E1,36 months,Source Verified,48000.0,0.0535,109.43,0.1864,3000,4,3939
2,1069243,CA,INDIVIDUAL,4 years,Chemat Technology Inc,C,RENT,05-01-2021,12-12-2021,09-01-2021,...,C5,36 months,Not Verified,50000.0,0.2088,421.65,0.1596,12000,11,3522
3,1041756,TX,INDIVIDUAL,< 1 year,barnes distribution,B,MORTGAGE,25-02-2021,12-12-2021,12-03-2021,...,B2,60 months,Source Verified,42000.0,0.0540,97.06,0.1065,4500,9,4911
4,1068350,IL,INDIVIDUAL,10+ years,J&J Steel Inc,A,MORTGAGE,01-01-2021,14-12-2021,15-01-2021,...,A1,36 months,Verified,83000.0,0.0231,106.53,0.0603,3500,28,3835


# Типы признаков:

In [8]:
df.dtypes.reset_index().rename(columns={"index":"feature", 0:"dtype"})


,feature,dtype
0,id,int64
1,address_state,object
2,application_type,object
3,emp_length,object
4,emp_title,object
5,grade,object
6,home_ownership,object
7,issue_date,object
8,last_credit_pull_date,object
9,last_payment_date,object


# Кол-во пропусков и уникальных значений:

In [9]:
info = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "missing": df.isna().sum(),
    "missing_%": (df.isna().mean()*100).round(2),
    "n_unique": df.nunique(dropna=False),
}).sort_values("missing_%", ascending=False)

info

,dtype,missing,missing_%,n_unique
emp_title,object,1438,3.73,28526
id,int64,0,0.00,38576
application_type,object,0,0.00,1
address_state,object,0,0.00,50
emp_length,object,0,0.00,11
grade,object,0,0.00,7
home_ownership,object,0,0.00,5
issue_date,object,0,0.00,65
last_credit_pull_date,object,0,0.00,107
last_payment_date,object,0,0.00,102


- пропуски почти отсутствуют, заметно только emp_title ~3.73%;
- emp_title имеет очень много уникальных значений (десятки тысяч). Его нельзя просто one-hot’нуть;
- application_type константный => можно смело удалять;
- если прогноз на момент выдачи, то убираем всё, что появляется после выдачи кредита (total_payment, last_payment_date, next_payment_date, last_credit_pull_dat);
- при временных данных нельзя ограничиваться train/test-разбиением, будем использовать out-of-time сплит;
- можно спокойно удалить следующие неинформативные столбцы: id, member_id.
- можно оставить только grade, а sub_grade удалить, так как два признака про одно и то же, сильная корреляция.

# Анализ таргета:

In [14]:
df["loan_status"].value_counts(dropna=False)

,count
loan_status,
Fully Paid,32145
Charged Off,5333
Current,1098


In [20]:
df_bin = df[df["loan_status"].isin(["Fully Paid", "Charged Off"])].copy()
df_bin["y"] = (df_bin["loan_status"] == "Charged Off").astype(int)
df_bin["y"].value_counts(normalize=True)

,proportion
y,
0,0.857703
1,0.142297


- loan_status — это дефолт/не дефолт, а не фрод в чистом виде, потому используем дефолт как proxy для потенциально мошеннических/рисковых заявок;
- у loan_status оставить только Fully Paid и Charged Off, а Current лучше исключить, потому что это «недонаблюдённый» статус;
- наблюдается дисбаланс классов: дефолтов ~14%;